In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.newswatch.kr/news/articleList.html?page={}&total=41000&box_idxno=&view_type=sm"
article_base = "https://www.newswatch.kr/"

dict = defaultdict(list)
for page in range(1, 2):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출

    ul_elem = driver.find_element(By.CSS_SELECTOR, 'ul.type2')
    li_list = ul_elem.find_elements(By.TAG_NAME, 'li')
    print(li_list)
    for li in li_list:
        a_tag = li.find_element(By.CSS_SELECTOR, 'a')
        href = a_tag.get_attribute('href')
        web_id = href.split('=')[1]
        article_url = 'https://www.newswatch.kr/news/articleView.html?idxno=' + web_id

        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', article_url)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        dict['Category'].append(article_soup.select_one('#article-view > div > header > nav > ul > li:nth-child(3) > a').get_text(strip=True))
        dict['Title'].append(article_soup.select_one('#article-view > div > header > h3').get_text(strip=True))
        
        date_element = article_soup.select_one("ul.infomation li:contains('입력')")
        if date_element:
            full_text = date_element.get_text(strip=True)  # '기사입력 2025-11-27 19:46:50'
            date_part = full_text.split()[1]  # '2025-11-27'
            date_formatted = date_part.replace('.', '-')
            dict['Date'].append(date_formatted)
        else:
            dict['Date'].append('')  # 또는 '날짜 없음'


        sentences = []

        atc = article_soup.select_one('#article-view-content-div')
        p_tags = atc.find_all('p')

        for p_tag in p_tags:
            text = p_tag.get_text(strip=True)
            trimmed = text[17:] if len(text) > 17 else ''
            sentences.append(trimmed)
        dict['Contents'].append(' '.join(sentences))


        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()



print(dict)

[<selenium.webdriver.remote.webelement.WebElement (session="d24476bc0a1dadd8e136d7e06414e3a5", element="f.B637A966E24EB4EC9259A0AF7296E8BB.d.7123AEC9A56435C78D48540ED25EB69F.e.830")>, <selenium.webdriver.remote.webelement.WebElement (session="d24476bc0a1dadd8e136d7e06414e3a5", element="f.B637A966E24EB4EC9259A0AF7296E8BB.d.7123AEC9A56435C78D48540ED25EB69F.e.840")>, <selenium.webdriver.remote.webelement.WebElement (session="d24476bc0a1dadd8e136d7e06414e3a5", element="f.B637A966E24EB4EC9259A0AF7296E8BB.d.7123AEC9A56435C78D48540ED25EB69F.e.851")>, <selenium.webdriver.remote.webelement.WebElement (session="d24476bc0a1dadd8e136d7e06414e3a5", element="f.B637A966E24EB4EC9259A0AF7296E8BB.d.7123AEC9A56435C78D48540ED25EB69F.e.862")>, <selenium.webdriver.remote.webelement.WebElement (session="d24476bc0a1dadd8e136d7e06414e3a5", element="f.B637A966E24EB4EC9259A0AF7296E8BB.d.7123AEC9A56435C78D48540ED25EB69F.e.873")>, <selenium.webdriver.remote.webelement.WebElement (session="d24476bc0a1dadd8e136d7e06

In [13]:
import json

articles = []
for i in range(len(dict['Category'])):
    article = {
        "num": i+1,
        "Category": dict['Category'][i],
        "Date": dict['Date'][i],
        "Title": dict['Title'][i],
        "Contents": dict['Contents'][i]
    }
    articles.append(article)

# JSON으로 저장
with open('articles_newswatch.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)
